# Exercise: Implement the A-Priori algorithm

Implement a version of the A-Priori algorithm on your own. You may assume your data is given as a list of baskets.

In [ ]:
import pandas as pd
import numpy as np

In [6]:
### read a csv file and extract the shopping items list

df = pd.read_csv('Groceries_dataset.csv')
baskets = [list(set(a[1]['itemDescription'].tolist())) for a in list(df.groupby('Member_number'))]
# baskets

### total size of baskets

# len(list(np.concatenate(baskets).flat))

In [7]:
### unique #items

items = set(list(np.concatenate(baskets).flat))
len(items)

167

In [8]:
### hash all singletons
df_item_hash = pd.DataFrame(range(len(items)), index = list(items), columns =['hashcode'], dtype=int)
df_item_hash

,hashcode
detergent,0
bottled water,1
meat,2
pip fruit,3
sweet spreads,4
...,...
other vegetables,162
cleaner,163
bottled beer,164
frozen fish,165


In [9]:
### count the items, store the count into the hashed array index

# item_count = pd.DataFrame(np.zeros((len(items),1)), index = list(items), columns =['count'], dtype=int)
item_count_arr = np.zeros((len(items),1))

for b in baskets:
    for item in b:
            idx = df_item_hash.loc[item,'hashcode']
            item_count_arr[idx] += 1

### find frequent items with support > s1 (here s1 = 0.02), and hash back from array index to items
freq_items  = [df_item_hash[df_item_hash['hashcode']==x].index[0] for x in np.where(item_count_arr > 0.02*len(baskets))[0]]
freq_items

# item_count_arr[item_count['count']>0.02*len(baskets)]
#freq_items['hashcode'] = list(range(1,len(freq_items)+1))

['detergent',
 'bottled water',
 'meat',
 'pip fruit',
 'butter milk',
 'oil',
 'salty snack',
 'sugar',
 'dishes',
 'frozen dessert',
 'yogurt',
 'frankfurter',
 'ice cream',
 'domestic eggs',
 'mustard',
 'hard cheese',
 'hygiene articles',
 'frozen meals',
 'turkey',
 'newspapers',
 'root vegetables',
 'canned fish',
 'sliced cheese',
 'curd',
 'UHT-milk',
 'chicken',
 'long life bakery product',
 'pet care',
 'rolls/buns',
 'pork',
 'fruit/vegetable juice',
 'seasonal products',
 'coffee',
 'baking powder',
 'roll products ',
 'grapes',
 'herbs',
 'processed cheese',
 'condensed milk',
 'cream cheese ',
 'canned vegetables',
 'onions',
 'ham',
 'pastry',
 'pasta',
 'margarine',
 'hamburger meat',
 'whipped/sour cream',
 'citrus fruit',
 'canned beer',
 'tropical fruit',
 'pickled vegetables',
 'frozen vegetables',
 'waffles',
 'chewing gum',
 'shopping bags',
 'white bread',
 'soda',
 'packaged fruit/vegetables',
 'flour',
 'liquor',
 'beef',
 'chocolate',
 'misc. beverages',
 'bro

In [10]:
### hash the frequent items (starting from 1)

df_freq_item_hash = pd.DataFrame(range(1,len(freq_items)+1), index=freq_items, columns=['hashcode'])
df_freq_item_hash

,hashcode
detergent,1
bottled water,2
meat,3
pip fruit,4
butter milk,5
...,...
specialty chocolate,83
other vegetables,84
bottled beer,85
frozen fish,86


In [ ]:
### triangular array encode function, (not used)
# def triangular_encode(i,j,n):
#     return int((i-1)*(n-i/2)+j-i)

In [11]:
### count the pairs using only frequent items, store the count into the (triangular) matrix.

# pair_mat = pd.DataFrame(np.zeros((len(freq_items.index),len(freq_items.index))),
#                         columns=freq_items.index, index=freq_items.index,
#                        dtype=int)

pair_mat_hashed = np.zeros((len(freq_items)+1,len(freq_items)+1))
# n = len(freq_items)
# triangular_arr = np.zeros((n*n,))


for b in baskets:
    cand_list = [item for item in b if item in freq_items]
    if len(cand_list)<2:
        continue
    for idx, item1 in enumerate(cand_list):
        for item2 in cand_list[idx+1:]:
            i = df_freq_item_hash.loc[item1,'hashcode']
            j = df_freq_item_hash.loc[item2,'hashcode']
            #triangular_arr[triangular_encode(i,j,n)] +=1
            #pair_mat.loc[item1, item2] += 1
            pair_mat_hashed[max(i,j),min(i,j)]+=1

# pair_mat
pair_mat_hashed

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,  29.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,  28., 157., ...,   0.,   0.,   0.],
       [  0.,   4.,  20., ...,  19.,   0.,   0.],
       [  0.,   5.,  31., ...,  21.,   5.,   0.]])

In [12]:
### extract frequent pairs that exceed support s2 (assume s2 = 0.02), and hash back.

freq_pairs = [[df_freq_item_hash[df_freq_item_hash['hashcode']==x].index[0], df_freq_item_hash[df_freq_item_hash['hashcode']==y].index[0]] for x, y in zip(*np.where(pair_mat_hashed > 0.02*len(baskets)))]
# freq_pairs
# freq_pairs = [[freq_itemset[x], freq_itemset[y]] for x, y in zip(*np.where(pair_mat.values > 0.02*len(baskets)))]

In [13]:
len(freq_pairs)

499

# Exercise 3: Use built in tools
Use/import the following Python packages: Pandas and MLxtend.  
Especially, have a look at apriori and association rules from mlxtend.frequent patterns.  
For documentation see: http://rasbt.github.io/mlxtend/

If helpful / desirable you might also use TransactionEncoder from mlxtend.preprocessing to clean / prepare your data.

The task: determine:
1. the frequent pairs of items.
2. the association rules of high confidence with or w/o high lift.
3. (optional) the association rules of high confidence with or w/o high interest. (optional)

In [ ]:
# ! pip install mlxtend

In [14]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

**Solutions**  
1. The frequent pairs of items.  
**I'm mainly refering to this documentation examples:** [reference](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/#apriori-frequent-itemsets-via-the-apriori-algorithm)

In [15]:
te = TransactionEncoder()
te_ary = te.fit(baskets).transform(baskets)
df_one_hot = pd.DataFrame(te_ary, columns=te.columns_)
df_one_hot

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,False,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,True,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3893,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3894,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
3895,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3896,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,False,True,False


In [16]:
frq_items = apriori(df_one_hot, min_support = 0.02, use_colnames = True)
frq_items['length'] = frq_items['itemsets'].apply(lambda x: len(x))
frq_items

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets,length
0,0.078502,(UHT-milk),1
1,0.031042,(baking powder),1
2,0.119548,(beef),1
3,0.079785,(berries),1
4,0.062083,(beverages),1
...,...,...,...
889,0.027963,"(soda, yogurt, whole milk, other vegetables)",4
890,0.021293,"(tropical fruit, other vegetables, yogurt, who...",4
891,0.021036,"(sausage, soda, whole milk, rolls/buns)",4
892,0.022832,"(sausage, whole milk, yogurt, rolls/buns)",4


In [18]:
### reformat a little, to put the frozenset into lists

ml_freq_items = []
for i in frq_items[frq_items['length']==1].itemsets.values:
    ml_freq_items.extend(list(i))

ml_freq_pairs = []
for i in frq_items[frq_items['length']==2].itemsets.values:
    ml_freq_pairs.append(list(i))

### check if the frequent itemsets found by ourselves and mlxtend are the same
for i in ml_freq_items:
    if i not in freq_items:
        print(i)

len(ml_freq_pairs)==len(freq_pairs)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


True

2. the association rules of high confidence with or w/o high lift.  
[doc example](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/#association_rules-association-rules-generation-from-frequent-itemsets)

In [19]:
association_rules(frq_items, metric="confidence", min_threshold=0.6)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(beef, other vegetables)",(whole milk),0.050795,0.458184,0.030528,0.601010,1.311723,0.007255,1.357970,0.250361
1,"(beef, rolls/buns)",(whole milk),0.040534,0.458184,0.024371,0.601266,1.312281,0.005800,1.358841,0.248021
2,"(beef, root vegetables)",(whole milk),0.033094,0.458184,0.020010,0.604651,1.319670,0.004847,1.370477,0.250526
3,"(beef, yogurt)",(whole milk),0.038481,0.458184,0.023602,0.613333,1.338619,0.005970,1.401249,0.263085
4,"(pastry, bottled beer)",(whole milk),0.033607,0.458184,0.020267,0.603053,1.316183,0.004869,1.364960,0.248581
5,"(rolls/buns, bottled beer)",(whole milk),0.063109,0.458184,0.038225,0.605691,1.321939,0.009309,1.374091,0.259940
6,"(shopping bags, bottled beer)",(whole milk),0.030272,0.458184,0.020010,0.661017,1.442690,0.006140,1.598358,0.316429
7,"(bottled water, brown bread)",(whole milk),0.037968,0.458184,0.023345,0.614865,1.341962,0.005949,1.406821,0.264879
8,"(bottled water, curd)",(whole milk),0.033094,0.458184,0.020010,0.604651,1.319670,0.004847,1.370477,0.250526
9,"(bottled water, yogurt)",(whole milk),0.066444,0.458184,0.040277,0.606178,1.323001,0.009833,1.375788,0.261519


In [20]:
association_rules(frq_items, metric="lift", min_threshold=1.2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(UHT-milk),(bottled water),0.078502,0.213699,0.021293,0.271242,1.269268,0.004517,1.078960,0.230217
1,(bottled water),(UHT-milk),0.213699,0.078502,0.021293,0.099640,1.269268,0.004517,1.023477,0.269801
2,(UHT-milk),(other vegetables),0.078502,0.376603,0.038994,0.496732,1.318979,0.009430,1.238697,0.262440
3,(other vegetables),(UHT-milk),0.376603,0.078502,0.038994,0.103542,1.318979,0.009430,1.027933,0.387936
4,(beef),(butter),0.119548,0.126475,0.020523,0.171674,1.357372,0.005403,1.054566,0.299031
...,...,...,...,...,...,...,...,...,...,...
1669,"(rolls/buns, whole milk)","(soda, yogurt)",0.178553,0.097486,0.024628,0.137931,1.414882,0.007222,1.046916,0.356964
1670,"(yogurt, rolls/buns)","(soda, whole milk)",0.111339,0.151103,0.024628,0.221198,1.463889,0.007804,1.090004,0.356590
1671,(whole milk),"(soda, yogurt, rolls/buns)",0.458184,0.042329,0.024628,0.053751,1.269836,0.005233,1.012071,0.392193
1672,(yogurt),"(soda, whole milk, rolls/buns)",0.282966,0.065162,0.024628,0.087035,1.335684,0.006190,1.023959,0.350499
